# DAGs in Snowflake
Let's say we have multiple notebooks that we want to schedule with a dependency graph. One option would be to create tasks, but they may be a bit granular and hard to visualize in a connected DAG via code. Snowflake offers a dag library where we can create this dependency graph via code.

In this example, we'll assume something simple - two notebooks where one needs to be run after the other. The notebooks don't really do anything (just printing characters), but the focus here is just on orchestration. 

We need to include the snowflake library as a package dependency in the notebook which allows us to execute these APIs. High level: 
1. Imports and grab session/root instances.
2. Create our dag with a context manager. Deploy to the same database/schema where the etl notebooks live.
3. Interactively test - grab the task instance and kick it off. In a production environment we'd have this scheduled with cron. 

## Libraries and Imports

In [ ]:
from snowflake.snowpark import Session
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.core import CreateMode
from snowflake.core import Root

In [ ]:
# Get active session in notebook
session = get_active_session()
root = Root(session)

## Define Dag

In [ ]:
with DAG(
    name='dag_orchestrator',
    warehouse='COMPUTE_WH'
) as dag:
    task_a = DAGTask(
        name="task_a",
        definition="EXECUTE NOTEBOOK TASK_A();"
    )
    task_b = DAGTask(
        name="task_b",
        definition="EXECUTE NOTEBOOK TASK_B();"
    )
    task_a >> task_b  # Dependency: task_b waits for task_a success

## Deploy and single test

In [ ]:
dag_op = DAGOperation(root.databases['LEARNING_DB'].schemas['ETL'])
dag_op.deploy(dag, mode=CreateMode.or_replace)

In [ ]:
tasks = root.databases["LEARNING_DB"].schemas["ETL"].tasks
dag_res = tasks['dag_orchestrator']
dag_res.execute()

In [ ]:
SELECT * FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(TASK_NAME => 'dag_orchestrator')) ORDER BY SCHEDULED_TIME DESC